<a href="https://colab.research.google.com/github/Mojn2000/DeepProject/blob/main/RNN_v1_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip3 install pickle5
import pickle5 as pickle
import pandas as pd
import numpy as np
import os
from sklearn.model_selection import train_test_split

from google.colab import drive
drive.mount('/content/drive')

drive_path = 'drive/My Drive/Skole/Uni/9_semester/02456_deep_learning/Project/'

if (not(os.path.exists(drive_path))):
  drive_path = 'drive/MyDrive/DL project/'  # Idas path    

f0 = "padded_data.pickle"

## LOAD PADDED and CLEANED data 
with open(drive_path + f0, 'rb') as f:
    data = pickle.load(f)


     |████████████████████████████████| 256 kB 4.3 MB/s 
Mounted at /content/drive


In [ ]:
# Load functions
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.optim as optim
from torch.nn import Linear, GRU, Conv2d, Dropout, MaxPool2d, BatchNorm1d
from torch.nn.functional import relu, elu, relu6, sigmoid, tanh, softmax, leaky_relu, linear 
from torch.utils.data import DataLoader
from torch.utils.data.sampler import SubsetRandomSampler

Separate into test, train, and bumpy roads 

In [ ]:
# Definition of bumpy road in IRI terms 
tau = 4.5 

good_data = data[data['IRI_mean'] <= tau]
bad_data = data[data['IRI_mean'] > tau]

# Separate good_data into test and train sets 
good_train, good_test = train_test_split(good_data, test_size=0.2)


# Build the model


In [ ]:
train_data = []
test_data = []
test_bad = []

batch_size = 32

# Test data
for i in range(len(good_test['GM.acc.xyz.z'])):
   x_data = good_test['GM.acc.xyz.z'].values[i].flatten()
   labels = [i] * len(x_data)
   test_data.append([x_data, labels])

test_loader = torch.utils.data.DataLoader(test_data, shuffle=True, batch_size=batch_size)

# Train data
for i in range(len(good_train['GM.acc.xyz.z'])):
   x_data = good_train['GM.acc.xyz.z'].values[i].flatten()
   labels = [i] * len(x_data)
   train_data.append([x_data, labels])

train_loader = torch.utils.data.DataLoader(train_data, shuffle=True, batch_size=batch_size)


# Bad data
for i in range(len(bad_data['GM.acc.xyz.z'])):
   x_data = bad_data['GM.acc.xyz.z'].values[i].flatten()
   labels = [i] * len(x_data)
   test_bad.append([x_data, labels])

bad_loader = torch.utils.data.DataLoader(test_bad, shuffle=True, batch_size=batch_size)


# RNN network

In [ ]:
# Define size variables

num_features = len(good_train['GM.acc.xyz.z'].values[0])
hidden_units = 24 
latent_features = 8  
layer_dim = 2 
dropout_prob = 0.1 #0.2


class AutoEncoder(nn.Module):
    def __init__(self, hidden_units, latent_features):
        super(AutoEncoder, self).__init__()

        # ENCODER
        self.rnn_in = nn.GRU(num_features,(hidden_units*1), layer_dim, batch_first=True, dropout=dropout_prob)
        self.linear_in2 = nn.Linear(in_features=hidden_units, out_features=latent_features)
        
        # ------------------
        # BOTTLE NECK
        # ------------------  

        # DECODER
        self.rnn_out = nn.GRU((latent_features), (hidden_units*1),layer_dim,batch_first=True, dropout=dropout_prob)
        self.linear_out0 = nn.Linear(in_features=(hidden_units*1), out_features=num_features,bias=True)


    def forward(self, x): 
        outputs = {}
        
        # RNN
        x = x[None,:, :]
        x, h = self.rnn_in(x)

        # LINEAR
        x = x[0, :, :]
        x = self.linear_in2(x)
        z = x

        # ------------------
        # BOTTLE NECK
        # ------------------        
        
        x = x[None,:, :]

        # RNN
        x, h = self.rnn_out(x)
        x = x[0, :, :]

        # LINEAR
        x = self.linear_out0(x)
        x_hat = x


        return {
            'z': z,
            'x_hat': x_hat
        }


# Choose the shape of the autoencoder
net = AutoEncoder(hidden_units=hidden_units, latent_features=latent_features)

print(net)

AutoEncoder(
  (rnn_in): GRU(3366, 24, num_layers=2, batch_first=True, dropout=0.1)
  (linear_in2): Linear(in_features=24, out_features=8, bias=True)
  (rnn_out): GRU(8, 24, num_layers=2, batch_first=True, dropout=0.1)
  (linear_out0): Linear(in_features=24, out_features=3366, bias=True)
)


## Set network parameters

In [ ]:
import torch.optim as optim

#OPTIMIZER
optimizer = torch.optim.RMSprop(net.parameters(), lr=1e-3, weight_decay=1e-4) 


# Custom loss function
def loss_function(x_hat,x):

    x_hat[x[:,:] == 0] = 0 

    loss = torch.diagonal(torch.matmul(x_hat - x,torch.transpose(x_hat - x ,0,1))) / torch.sum(x[:,:] != 0, dim = 1)
    #torch.sum((torch.abs(x_hat - x)), dim = 1) / torch.sum(x[:,:] != 0, dim = 1)

    return(sum(loss)/batch_size)


## Train the network 

In [ ]:
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA

train_loss = []
valid_loss = []

In [ ]:
# EPOCHS
num_epochs = 10

# TRAIN MODEL
for epoch in range(num_epochs):
    print("Epoch: ", epoch)
    batch_loss = []
    net.train()
    
    for x,y in train_loader:
        x = x.float()
        outputs = net(x)
        x_hat = outputs['x_hat']

        # LOSS
        loss = loss_function(x_hat, x)
        
        # UPDATE MODEL
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        batch_loss.append(loss.item())

    train_loss.append(np.mean(batch_loss))

    # Evaluate, do not propagate gradients
    with torch.no_grad():
        net.eval()
      
        x, y = next(iter(test_loader))
        x = x.float()
        outputs = net(x)

        x_hat = outputs['x_hat']
        z = outputs['z'].cpu().numpy()

        loss = loss_function(x_hat, x)

        valid_loss.append(loss.item())
    
    if epoch == 0:
        continue

In [ ]:
# PLOT TRAINING CURVE 
import matplotlib.pyplot as plt
len_epoch = range(len(train_loss))

plt.rcParams.update(plt.rcParamsDefault)
plt.figure(dpi=800)
plt.plot(len_epoch,np.log10(train_loss),color='C3',label="Train loss")
plt.plot(len_epoch,np.log10(valid_loss),color='b', label= "Validation loss")
plt.title("Train Loss")
plt.xlabel("Epochs")
plt.ylabel("log(MSE)")
plt.grid()
plt.legend()
plt.show()


In [ ]:
# PLOT TEST
sqerr_good = []
for x,y in test_loader:
  x = x.float()
  outputs = net(x)
  x_hat = outputs['x_hat']
  x_hat[x==0] = 0
  sqerr_good.extend((torch.diagonal(torch.matmul(x_hat - x,torch.transpose(x_hat - x ,0,1))) / torch.sum(x[:,:] != 0, dim = 1)).tolist())


# PLOT TRAIN 
sqerr_good_train = []
for x,y in train_loader:
  x = x.float()
  outputs = net(x)
  x_hat = outputs['x_hat']
  x_hat[x==0] = 0
  sqerr_good_train.extend((torch.diagonal(torch.matmul(x_hat - x,torch.transpose(x_hat - x ,0,1))) / torch.sum(x[:,:] != 0, dim = 1)).tolist())


# PLOT BAD 
sqerr_bad = []
for x,y in bad_loader:
  x = x.float()
  outputs = net(x)
  x_hat = outputs['x_hat']
  x_hat[x==0] = 0
  sqerr_bad.extend((torch.diagonal(torch.matmul(x_hat - x,torch.transpose(x_hat - x ,0,1))) / torch.sum(x[:,:] != 0, dim = 1)).tolist())


In [ ]:
# PLOTTING OF RESULTS 

plt.hist(sqerr_good, bins = 30, alpha=0.5, label='Test',density = True)
#plt.hist(sqerr_good_train, bins = 30, alpha=0.5, label='Train',density = True)
plt.hist(sqerr_bad, bins = 30, alpha=0.5, label='Bad',density = True)
plt.legend(loc='upper right')
plt.title("Network test for good and bad data")
plt.show()

#Compute performance

In [ ]:
from math import log10

eps = 1e-6
tau_min = min(min(sqerr_good),min(sqerr_bad)) - eps
tau_max = max(max(sqerr_good),max(sqerr_bad)) + eps
print(tau_min)
print(tau_max)

N = 100

perf = np.array([np.zeros(N), np.zeros(N), np.zeros(N), np.zeros(N), np.zeros(N)])
ii = 0
#for tau in np.linspace(tau_min, tau_max, num = N):
for tau in np.logspace(log10(tau_min), log10(tau_max), num = N):
  perf[0,ii] = tau
  perf[1,ii] = sum(sqerr_good<tau)  # true positives
  perf[2,ii] = sum(sqerr_good>=tau) # false positive 
  perf[3,ii] = sum(sqerr_bad<tau)   # false negatives
  perf[4,ii] = sum(sqerr_bad>=tau) # true negatives
  ii += 1

perf = pd.DataFrame(perf)
perf.to_pickle(drive_path+'RNN_v1_3.pickle')

6.217727820714936e-05
0.055485384298324586
